In [1]:
# conda list --export > requirements_conda.txt

In [2]:
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from tools.dset_getters import cvsi_dset, me_dset
from tools.utils import simple_coll, padding_coll_factory
# from tools.basic_model import NeuralNetwork

In [3]:
# a dummy model with an adjustable number of classes
class NeuralNetwork(nn.Module):
        
    def __init__(self, number_of_classes, vocab_size, embedding_dim, context_size):
        super(NeuralNetwork, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=16, kernel_size=8, bias=False)
        self.norm1 = nn.BatchNorm1d(16)
        self.relu = nn.ReLU()
        self.pool1 = nn.MaxPool1d(2)
        
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=8, kernel_size=8, bias=False)
        self.norm2 = nn.BatchNorm1d(8)
        self.pool2 = nn.MaxPool1d(2)

        self.conv3 = nn.Conv1d(in_channels=8, out_channels=4, kernel_size=8, bias=False)
        self.norm3 = nn.BatchNorm1d(4)
        self.pool3 = nn.MaxPool1d(2)

        self.conv4 = nn.Conv1d(in_channels=4, out_channels=3, kernel_size=8, bias=False)
        self.norm4 = nn.BatchNorm1d(3)
        self.pool4 = nn.MaxPool1d(2)
        
        self.flatten = nn.Flatten()
        self.lin1 = nn.Linear(861, 512)
        self.lin2 = nn.Linear(512, number_of_classes)
        self.softmax = nn.Softmax(dim=0)
        self.loss = torch.nn.functional.binary_cross_entropy_with_logits


        
#         self.linear1 = nn.Linear(context_size * embedding_dim, 128)
#         self.linear2 = nn.Linear(128, number_of_classes)
#         self.softmax = nn.Softmax(dim=0)
#         self.loss = torch.nn.functional.binary_cross_entropy_with_logits
        
#         self.conv1 = nn.Conv1d(in_channels=input_len, out_channels=n_hidden, kernel_size=kernel_size, bias=False),
#             nn.BatchNorm1d(n_hidden),
#             nn.ReLU(inplace=True),

    def forward(self, x):
#         print("start ", x.size())
        x = self.embeddings(x)
#         print("emb ", x.size())
        x = x.transpose(1, 2)
#         print("trans ", x.size())
        x = self.conv1(x)
#         print("conv ", x.size())
        x = self.norm1(x)
        x = self.relu(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.norm2(x)
        x = self.relu(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = self.norm3(x)
        x = self.relu(x)
        x = self.pool3(x)
        
        x = self.conv4(x)
        x = self.norm4(x) 
        x = self.relu(x)
        x = self.pool4(x)
        
        x = self.flatten(x)
        x = self.lin1(x) 
        x = self.lin2(x) 
        x = self.softmax(x)
        return x
        
#         print("start ", x.size())
# #         x = self.embeddings(x).view((1, -1))
#         x = self.embeddings(x)
#         print("emb ", x.size())
#         x = self.linear1(x)
#         print("lin1 ", x.size())
#         x = self.relu(x)
#         print("relu ", x.size())
#         x = self.linear2(x)
#         print("lin2 ", x)
#         x = self.softmax(x)
#         print("soft ", x.size())
#         return x
        
    def train_loop(self, dataloader, optimizer):
        for x, y in dataloader:
            pred = model(x)
#             print('pred: ', pred)
#             print('y: ', y)
            loss = self.loss(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
    def train(self, dataloader, epochs):
        learning_rate = 1e-3
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
        for t in range(epochs):
            print(f"Epoch {t}")
            model.train_loop(dataloader, optimizer)

    def test(self, dataloader):
        size = dataloader.dataset.__len__()
        num_batches = len(dataloader)
        test_loss, correct = 0, 0

        with torch.no_grad():
            for X, y in dataloader:
                pred = model(X)
                test_loss += self.loss(pred, y).item()
                correct += (torch.round(pred) == y).sum().item()

        print('test_loss ', test_loss)
        print('num_batches', num_batches)
        print('correct', correct)
        print('size', size)

        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

get_dataset_fn = me_dset
train_dset = get_dataset_fn('train', force_download=False)


Using cuda device
Reference fasta_cache/Mus_musculus.GRCm38.dna_rm.toplevel.fa.gz already exists. Skipping.


/home/jupyter/genomic_benchmarks/tools/loc2seq/with_biopython.py:77: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


  0%|          | 0/21 [00:00<?, ?it/s]

## Tokenizer and vocab

In [5]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import vocab, build_vocab_from_iterator
from collections import Counter 

class LetterTokenizer():
    def __init__(self, **kwargs):
        pass
    def __call__(self, items):
        if isinstance(items, str):
            return self.__tokenize_str(items)
        else:
            return (self.__tokenize_str(t) for t in items)
    def __tokenize_str(self, t):
        tokenized = list(t.replace("\n",""))
        tokenized.append('<eos>')
        tokenized.insert(0,'<bos>')
        return tokenized

tokenizer = get_tokenizer(LetterTokenizer())
# (xx, yy) = zip(*train_dset)

def build_vocab(dataset, tokenizer):
    counter = Counter()
    for i in range(len(dataset)):
        counter.update(tokenizer(dataset[i][0]))
    print(counter.most_common())
    builded_voc = vocab(counter)
    builded_voc.append_token('<pad>')
    return builded_voc

# todo: why build fn does not work as expected (iterator argument)
#     return build_vocab_from_iterator(
#         iterator = counter, 
#         specials = ['<unk>', '<pad>', '<bos>', '<eos>'],
#         special_first = True)

vocabulary = build_vocab(train_dset, tokenizer)
print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

[('N', 694058), ('T', 429929), ('A', 429509), ('C', 356462), ('G', 352072), ('<bos>', 968), ('<eos>', 968)]
vocab len: 8
{'N': 5, '<pad>': 7, 'T': 3, 'C': 4, 'A': 2, '<eos>': 6, 'G': 1, '<bos>': 0}


In [6]:
# tokenizer(train_dset[0][0])
# train_dset[0][0]

In [7]:
# create padding collate 
input_len = max([len(train_dset[i][0]) for i in range(len(train_dset))])
print(input_len)
# padding_coll_factory(longest_length, vocab, tokenizer):
collate = padding_coll_factory(input_len, vocabulary, tokenizer)

train_loader = DataLoader(train_dset, batch_size=32, shuffle=True, collate_fn=collate)


4707


In [8]:
# print(next(iter(train_loader))[0][0])
# print(next(iter(train_loader))[1][0])

In [9]:
# __init__(self, number_of_classes, vocab_size, embedding_dim, context_size):
model = NeuralNetwork(
    number_of_classes=1, 
    vocab_size=vocabulary.__len__(),
    embedding_dim=5,
    context_size=input_len
).cuda()
model.train(train_loader, epochs=50)

Epoch 0


/home/jupyter/genomic_benchmarks/tools/utils.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(tmp, dtype=torch.long)
/opt/conda/envs/bench_env/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49


In [10]:
test_dset = get_dataset_fn('test', force_download=False)
test_loader = DataLoader(test_dset, batch_size=32, shuffle=True, collate_fn=collate)
model.test(test_loader)

Reference fasta_cache/Mus_musculus.GRCm38.dna_rm.toplevel.fa.gz already exists. Skipping.


  0%|          | 0/21 [00:00<?, ?it/s]

test_loss  5.510492742061615
num_batches 8
correct 121
size 242
Test Error: 
 Accuracy: 50.0%, Avg loss: 0.688812 



In [12]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

embeddings.weight True
conv1.weight True
norm1.weight True
norm1.bias True
conv2.weight True
norm2.weight True
norm2.bias True
conv3.weight True
norm3.weight True
norm3.bias True
conv4.weight True
norm4.weight True
norm4.bias True
lin1.weight True
lin1.bias True
lin2.weight True
lin2.bias True
